In [ ]:
# Tile names from AHN retriever
from osgeo import ogr, osr
import requests
import os

url = 'https://basisdata.nl/hwh-ahn/AUX/bladwijzer_AHN6.gpkg' 
gpkg_filename = "bladwijzer_AHN6.gpkg"

try:
    response = requests.get(url, stream=True)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    with open(gpkg_filename, 'wb') as file:
        file.write(response.content)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

driver = ogr.GetDriverByName("GPKG")
dataSource = driver.Open(gpkg_filename, 0)
layer_count = dataSource.GetLayerCount()
print(f"The GeoPackage contains {layer_count} layers.")

for i in range(layer_count):
    layer = dataSource.GetLayerByIndex(i)
    layer_name = layer.GetName()
    feature_count = layer.GetFeatureCount()    
    print(f"  - Layer {i}: {layer_name} (Features: {feature_count})")

# Store the tile names
tile_names = []
layer = dataSource.GetLayerByIndex(0) 
for feature in layer:
    field_value = feature.GetField(3)
    tile_names.append(field_value)
print(f"retrieved {len(tile_names)} tiles")

os.remove(gpkg_filename)

# 2) Compare the list of desired files to process with the processed files in MinIO
# 3) retain from the list the unprocessed files

# Randomize the order of the list


In [ ]:
import random
random.shuffle(tile_names)

param_max_batch_count = 10

# group items into batches
def batch_items(items, max_batch_count) -> list:
    if len(items) == 0:
        return [[]]
    batch_size = len(items) // max_batch_count + bool(len(items) % max_batch_count)
    return [items[i:i+batch_size] for i in range(0, len(items), batch_size)]

batches = batch_items(tile_names, param_max_batch_count)



In [ ]:
# The format of the file https://basisdata.nl/hwh-ahn/AHN6/01_LAZ/AHN6_2025_C_277000_581000.LAZ